# Generate a single field primary beam

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/cube_single_field_primary_beam.ipynb)



This notebook domonstrates how to generate a primary beam image with provided some imaging parameters

---

## Assumptions/Current limitation


- Currently only support ALMA telescopes (ALMA 12m and ACA 7m)
- Only airy disk model with a blockage is supported
- Following parameters are used :
  * Effective dish diameter: ALMA 10.7m,  ACA 6.7m    
  * Blockage diameter: ALMA 0.75m, ACA 0.75m

---
#### Obsercured airy disk model
##### The intensity distrubiton for casa_airy_disk is defined as follow,
$$ I(x)/I_0 = \left[ \frac{1}{(area\_ratio - 1)}\left( \frac{area\_ratio * 2 * J_1(x)}{x}
                   - \frac{2 * J_1(x * length\_ratio)}{(x * length\_ratio)}
                \right)
            \right] ^ 2 $$
            where $ length\_ratio = \frac{effective\_diameter}{blockage\_diameter} $ and 
            $ area\_ratio = (length\_ratio)^2 $.


##### The intensity distribution for airy_disk is defined as follow,
            
$$I(x)/I_0 = \left[ \frac{1}{1 - e^2} \left( \frac{2 * J_1(x)}{x} - \frac{2 * e * J_1(x * e)}{x} \right) \right]^2$$
                where $ e = blockage\_diameter/effective\_diameter $.

So, **the casa airy disk model** above can be written in terms of the scaling factor, $e$,

$$ I(x)/I_0 = \left[ \frac{1}{1 - e^2}\left( \frac{ 2 * J_1(x)}{x}
                   - \frac{2 * e^3 * J_1(x / e)}{x}
                \right)
            \right] ^ 2 $$

In the CASA airy disk model, in the term accounting for the blockage has the extra factor of $e^2$ and scaling inside Bessel function is inverted as compared the standard expression for the obsecured airy disk. From the theoretical point of view, the CASA version of the equation seems to be wrong but the numerical differences, especially for small blockage ratio and well within the accuracy of the actural ALMA primary beam determination. The numerical differences are discussed in more detail the last section of this notebook.

## Install AstroVIPER

Skip this cell if you don't want to install the latest version of AstroVIPER.

In [ ]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

## API

In [ ]:
from astroviper.core.imaging.imaging_utils.make_primary_beam import cube_single_field_primary_beam

In [ ]:
cube_single_field_primary_beam?

## Import additional modules needed for this demo

In [ ]:
from xradio.image import open_image, load_image
from toolviper.utils.data import download, update
import numpy as np

## Example 1
Generate ALMA (12m) primary beam and compare with the PB image made by CASA

### Download Data

In [ ]:
# Turn this on when the data is available at the toolviper data repo.
update()
# Download the CASA PB image to compare with 
download(file="3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb")

In [ ]:
# Load CASA pb image to be used for comparison
casapb_im = open_image(
        "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb",
)
casapb_im

### Construct imaging parameters based on the CASA PB image

In [ ]:
# Extract some image parameters 
phase_center = casapb_im.attrs['coordinate_system_info']['reference_direction']['data']
image_shape = casapb_im.PRIMARY_BEAM.shape
cell_size = [(casapb_im.l[1]-casapb_im.l[0]).values.item(), (casapb_im.m[1]-casapb_im.m[0]).values.item()]
frequency_coords = casapb_im.frequency.values.tolist()
polarization_coords = casapb_im.polarization.values.tolist()
time_coords = casapb_im.time.values.tolist()

In [ ]:
# Store in imaging parameter dictionary 
im_params={}
 
im_params['image_size'] = image_shape[-2:]
im_params['cell_size'] = tuple(cell_size)
im_params['image_center'] = tuple([int(im_params['image_size'][0]/2), int(im_params['image_size'][1]/2)])
im_params['phase_center'] = tuple(phase_center)
im_params['frequency_coords'] = frequency_coords
im_params['polarization'] = polarization_coords
im_params['time_coords'] = time_coords

### Generate PB image

In [ ]:
pb_image = cube_single_field_primary_beam(im_params, telescope='ALMA')

In [ ]:
# select single timestamp, polarization, frequency (use channel 0 data)
data_sel = {"time": [0], "polarization": [0], "frequency": [4]}

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
im = pb_image["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.pcolormesh(x='right_ascension', y='declination')
ct = pb_image["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.contour(levels=10, colors='white', linewidths=0.5, x='right_ascension', y='declination')
plt.title('Astroviper model=casa_airy_disk (channel 0)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))


im = casapb_im["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.pcolormesh(x='right_ascension', y='declination')
ct = casapb_im["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.contour(levels=10, colors='white', linewidths=0.5, x='right_ascension', y='declination')
plt.title('CASA pb image')
plt.show()

In [ ]:
diff_ch0_im = pb_image["PRIMARY_BEAM"].isel(data_sel).data.squeeze() - casapb_im["PRIMARY_BEAM"].isel(data_sel).data.squeeze()


In [ ]:
rel_diff = abs(diff_ch0_im.compute())/casapb_im["PRIMARY_BEAM"].isel(data_sel).data.squeeze().compute()

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(rel_diff)
plt.colorbar()
plt.title('Fractional differences between CASA PB image and Astroviper image')

In [ ]:
# Plot histogram of fractional differences
plt.figure(figsize=(8,6))
flatten_data = rel_diff.ravel()
plt.hist(flatten_data)

### Airy disk model based on the equation from the Wikipedia 
model = 'airy_model' uses model based on the equation for Obecured Airy Pattern in https://en.wikipedia.org/wiki/Airy_disk

In [ ]:
pb_image2 = cube_single_field_primary_beam(im_params, telescope='ALMA', model='airy_model')

In [ ]:
plt.figure(figsize=(8,6))
im = pb_image2["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.pcolormesh(x='right_ascension', y='declination')
ct = pb_image2["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.contour(levels=10, colors='white', linewidths=0.5, x='right_ascension', y='declination')
plt.title('Model=airy_disk (obsecured airy disk model from Wikipedia')
plt.show()

In [ ]:
diff_ch0_im2 = pb_image2["PRIMARY_BEAM"].isel(data_sel).data.squeeze() - casapb_im["PRIMARY_BEAM"].isel(data_sel).data.squeeze()
rel_diff2 = abs(diff_ch0_im2.compute())/casapb_im["PRIMARY_BEAM"].isel(data_sel).data.squeeze().compute()

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(rel_diff2)
plt.colorbar()
plt.title('Fractional differences between CASA PB and astroviper PB using airy_disk model (ch0 only)')
plt.show()

## Example 2

Generate ACA 7-m PB image


In [ ]:
acapb_image = cube_single_field_primary_beam(im_params, telescope='ACA')

In [ ]:
plt.figure(figsize=(8,6))
im = acapb_image["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.pcolormesh(x='right_ascension', y='declination')
ct = acapb_image["PRIMARY_BEAM"].isel(data_sel).squeeze().plot.contour(levels=10, colors='white', linewidths=0.5, x='right_ascension', y='declination')
plt.show()

## Direct comparison of the two airy models
We explore here the numerical differences of the two airy disk equations noted in above. The differences are relatively small but slighly larger for ACA dish.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jn
from ipywidgets import interact, FloatSlider, Layout
import ipywidgets as widgets

# Evaluated at 230GHz
c = 3e8  # Speed of light (m/s)
freq = 230e9  # Frequency (Hz)
wavelength = c / freq  # Wavelength (m)


#### Function for airy disk models 

In [ ]:
def airy_pattern(r, D, wavelength, e=0, casa_model=False):
    """
    Calculate the Airy disk pattern with central obscuration.
    
    Parameters:
    r: radial distance (in units of theta)
    D: aperture diameter 
    wavelength: wavelength of light
    e: obscuration ratio (d/D where d is central blockage diameter)
    casa_model: if True, use x_obs = x/e with e**3 coefficient
    """
    #k = 2 * np.pi / wavelength
    x = np.pi * D * r / wavelength
    
    # Avoid division by zero at center
    x = np.where(x == 0, 1e-10, x)
    
    if e == 0 or e < 0.001:
        # Unobscured Airy pattern
        I = (2 * jn(1,x) / x)**2
    else:
        # Obscured Airy pattern
        if casa_model:
            # casa version: x_obs = x / e, with e**3 coefficient
            x_obs = x / e
            numerator = 2 * jn(1,x) - 2 * e**3 * jn(1,x_obs)
        else:
            # standard expression: x_obs = e * x, with e**2 coefficient
            x_obs = e * x
            numerator = 2 * jn(1,x) - 2 * e**2 * jn(1,x_obs)
            
        denominator = x * (1 - e**2)
        I = (numerator / denominator)**2
    
    return I/np.max(I)

#### Plotting function

In [ ]:
def plot_comparison(D=10.7, d=0.75):
    """
    Plot comparison for different aperture diameters and blockage diameters.
    
    Parameters:
    D: Aperture diameter in meters
    d: Central obscuration diameter in meters
    """
    # Calculate obscuration ratio
    e = d / D
    theta_res = 1.22 * wavelength / D  # radians
    
    # Check for valid parameters
    if d >= D:
        print(f"ERROR: Obscuration diameter (d={d}m) must be less than aperture diameter (D={D}m)")
        return
    
    # Create figure with 6 subplots (3 rows, 3 columns)
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.40, wspace=0.3, height_ratios=[1, 1, 1])
    
    # Top row: Full view
    ax1 = fig.add_subplot(gs[0, :])  # Full 1D profile
    ax2 = fig.add_subplot(gs[1, 0])  # standard 2D
    ax3 = fig.add_subplot(gs[1, 1])  # CASA 2D
    ax4 = fig.add_subplot(gs[1, 2])  # Difference 2D
    
    # Bottom row: Zoomed views
    ax5 = fig.add_subplot(gs[2, 0])  # Zoomed 1D - central peak
    ax6 = fig.add_subplot(gs[2, 1])  # Zoomed 1D - first ring
    ax7 = fig.add_subplot(gs[2, 2])  # Zoomed 2D difference
    
    # Create angular coordinate grid for 1D plot
    theta_max = 5 * theta_res  # Plot up to 5 Airy disk radii
    theta = np.linspace(0, theta_max, 1000)
    radtoarcsec = 180 / np.pi * 3600
    theta_arcsec = theta * radtoarcsec  # Convert to arcsec
    
    # Calculate 1D profiles -  standard and casa models
    I = airy_pattern(theta, D, wavelength, e, casa_model=False)
    I_casa = airy_pattern(theta, D, wavelength, e, casa_model=True)

        # ============= FULL 1D PROFILE =============
    ax1.plot(theta_arcsec, I, 'b-', linewidth=2.5, label='Standard: 2J₁(x)/x- 2e²J₁(ex)/ex')
    ax1.plot(theta_arcsec, I_casa, 'r--', linewidth=2.5, label='CASA: 2J₁(x) - 2e³J₁(x/e)')
    ax1.set_xlabel('Angular Distance (arcsec)', fontsize=12)
    ax1.set_ylabel('Normalized Intensity', fontsize=12)
    ax1.set_title(f'Full Airy Disk Profile | D={D:.2f}m, d={d:.2f}m, e={e:.4f} | f={freq/1e9:.0f}GHz, λ={wavelength*1e3:.2f}mm', 
                  fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.1)
    ax1.set_xlim(0, theta_arcsec[-1])
    ax1.legend(loc='upper right', fontsize=10)
    
    # ============= ZOOMED 1D - CENTRAL PEAK =============
    zoom_central = 0.5 * theta_res * radtoarcsec # First 50% of first Airy radius
    idx_central = np.where(theta_arcsec <= zoom_central)[0]
    
    ax5.plot(theta_arcsec[idx_central], I[idx_central], 'b-', linewidth=3, label='standard')
    ax5.plot(theta_arcsec[idx_central], I_casa[idx_central], 'r--', linewidth=3, label='CASA')
    ax5.set_xlabel('Angular Distance (arcsec)', fontsize=11)
    ax5.set_ylabel('Normalized Intensity', fontsize=11)
    ax5.set_title('ZOOMED: Central Peak Region', fontsize=11, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    ax5.legend(fontsize=9)
    
    # Add text showing peak values
    #peak_correct = I[0]
    #peak_wrong = I_casa[0]
    #peak_text = f'Peak values:\nCorrect: {peak_correct:.4f}\nWrong: {peak_wrong:.4f}\nDiff: {abs(peak_correct-peak_wrong):.4f}'
    #ax5.text(0.95, 0.95, peak_text, transform=ax5.transAxes, 
    #         fontsize=9, verticalalignment='top', horizontalalignment='right',
    #         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
       # ============= ZOOMED 1D - FIRST RING =============
    zoom_ring_start = 1.0 * theta_res * radtoarcsec
    zoom_ring_end = 3.0 * theta_res * radtoarcsec
    idx_ring = np.where((theta_arcsec >= zoom_ring_start) & (theta_arcsec <= zoom_ring_end))[0]
    
    if len(idx_ring) > 0:
        ax6.plot(theta_arcsec[idx_ring], I[idx_ring], 'b-', linewidth=3, label='standard')
        ax6.plot(theta_arcsec[idx_ring], I_casa[idx_ring], 'r--', linewidth=3, label='CASA')
        ax6.set_xlabel('Angular Distance (arcsec)', fontsize=11)
        ax6.set_ylabel('Normalized Intensity', fontsize=11)
        ax6.set_title('ZOOMED: First Ring Region', fontsize=11, fontweight='bold')
        ax6.grid(True, alpha=0.3)
        ax6.legend(fontsize=9)
    
    # Create 2D image coordinates
    extent_arcsec = theta_max * 180 / np.pi * 3600  # arcseconds
    n_pixels = 400
    x = np.linspace(-theta_max, theta_max, n_pixels)
    y = np.linspace(-theta_max, theta_max, n_pixels)
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X**2 + Y**2)
    
    # Calculate 2D images
    I_2d = airy_pattern(R, D, wavelength, e, casa_model=False)
    I_2d_casa = airy_pattern(R, D, wavelength, e, casa_model=True)
     # ============= 2D IMAGES =============
    # Plot 2D image - airy_disk model (standard equation)
    im1 = ax2.imshow(I_2d, extent=[-extent_arcsec, extent_arcsec, -extent_arcsec, extent_arcsec],
                     cmap='hot', origin='lower', interpolation='bilinear')
    ax2.set_xlabel('Arcseconds', fontsize=11)
    ax2.set_ylabel('Arcseconds', fontsize=11)
    if e == 0:
        ax2.set_title(f'airy_disk Model\nnumerator = 2J₁(x)', 
                  fontsize=10, color='blue', fontweight='bold')
    else:
        ax2.set_title(f'airy_disk Model\nnumerator = 2J₁(x) - 2e²J₁(ex)\ne²={e**2:.6f}, εx={e:.4f}x', 
                  fontsize=10, color='blue', fontweight='bold')
    plt.colorbar(im1, ax=ax2, label='Intensity')
    
    # Plot 2D image - casa_airy_disk model
    im2 = ax3.imshow(I_2d_casa, extent=[-extent_arcsec, extent_arcsec, -extent_arcsec, extent_arcsec],
                     cmap='hot', origin='lower', interpolation='bilinear')
    ax3.set_xlabel('Arcseconds', fontsize=11)
    ax3.set_ylabel('Arcseconds', fontsize=11)
    if e == 0:
        ax3.set_title(f'CASA Model\nnumerator = 2J₁(x)', 
                  fontsize=10, color='red', fontweight='bold')
    else:
        ax3.set_title(f'CASA Model\nnumerator = 2J₁(x) - 2e³J₁(x/e)\ne³={e**3:.6f}, x/ε={1/e:.2f}x', 
                  fontsize=10, color='red', fontweight='bold')
    plt.colorbar(im2, ax=ax3, label='Intensity')
        # Plot difference map
    diff = np.abs(I_2d - I_2d_casa)
    im3 = ax4.imshow(diff, extent=[-extent_arcsec, extent_arcsec, -extent_arcsec, extent_arcsec],
                     cmap='viridis', origin='lower', interpolation='bilinear')
    ax4.set_xlabel('Arcseconds', fontsize=11)
    ax4.set_ylabel('Arcseconds', fontsize=11)
    ax4.set_title('Absolute Difference\n|Standard - CASA|', fontsize=11, fontweight='bold')
    plt.colorbar(im3, ax=ax4, label='|ΔI|')
    
    # ============= ZOOMED 2D DIFFERENCE =============
    # Create zoomed coordinates for 2D (central region only - 2x Airy radius)
    theta_zoom = 2 * theta_res
    extent_zoom_arcsec = theta_zoom * 180 / np.pi * 3600
    n_pixels_zoom = 300
    x_zoom = np.linspace(-theta_zoom, theta_zoom, n_pixels_zoom)
    y_zoom = np.linspace(-theta_zoom, theta_zoom, n_pixels_zoom)
    X_zoom, Y_zoom = np.meshgrid(x_zoom, y_zoom)
    R_zoom = np.sqrt(X_zoom**2 + Y_zoom**2)
    
    I_2d_zoom = airy_pattern(R_zoom, D, wavelength, e, casa_model=False)
    I_2d_casa_zoom = airy_pattern(R_zoom, D, wavelength, e, casa_model=True)
    diff_zoom = np.abs(I_2d_zoom - I_2d_casa_zoom)
    
    im4 = ax7.imshow(diff_zoom, extent=[-extent_zoom_arcsec, extent_zoom_arcsec, 
                                         -extent_zoom_arcsec, extent_zoom_arcsec],
                     cmap='viridis', origin='lower', interpolation='bilinear')
    ax7.set_xlabel('Arcseconds', fontsize=11)
    ax7.set_ylabel('Arcseconds', fontsize=11)
    ax7.set_title('ZOOMED Difference\nCentral 2×', fontsize=11, fontweight='bold')
    plt.colorbar(im4, ax=ax7, label='|ΔI|')
    

#### Make plotting interactive

In [ ]:
# Create interactive widget with sliders
interact(plot_comparison, 
         D=FloatSlider(
             value=10.7,
             min=2.0,
             max=20.0,
             step=0.1,
             description='Effective diameter, D (m):',
             style={'description_width': 'initial'},
             layout=Layout(width='600px'),
             continuous_update=False
         ),
         d=FloatSlider(
             value=0.75,
             min=0.0,
             max=5.0,
             step=0.05,
             description='Blockage diameter, d (m):',
             style={'description_width': 'initial'},
             layout=Layout(width='600px'),
             continuous_update=False
         ))